__Описание проекта "Обучение с учителем: качество модели".__

   Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Необходимо разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.


__План выполнения:__

  1. Промаркировать уровень финансовой активности постоянных покупателей. В компании принято выделять два уровня активности: «снизилась», если клиент стал покупать меньше товаров, и «прежний уровень».
  
  2. Собрать данные по клиентам по следующим группам: 
 - Признаки, которые описывают коммуникацию сотрудников компании с клиентом.
 - Признаки, которые описывают продуктовое поведение покупателя. Например, какие товары покупает и как часто.
 - Признаки, которые описывают покупательское поведение клиента. Например, сколько тратил в магазине.
 - Признаки, которые описывают поведение покупателя на сайте. Например, как много страниц просматривает и сколько времени проводит на сайте.

 3. Построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца.


 4. В исследование включить дополнительные данные финансового департамента о прибыльности клиента: какой доход каждый покупатель приносил компании за последние три месяца.


 5. Используя данные модели и данные о прибыльности клиентов,выделить сегменты покупателей и разработать для них персонализированные предложения.

## __Шаг 1. Загрузка данных__

Загрузим необходимые библиотеки

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_classification
!pip install shap
import shap
RANDOM_STATE = 42
TEST_SIZE=0.25

  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/e3/94/ad3e57ddae693220c77852bb0b427b98dc5ba15c8da5202218adfb64b0ab/shap-0.47.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 322.0 kB/s eta 0:00:0000:0100:01


Загрузим данные, и ознакомимся с ними.

In [4]:
market_file = pd.read_csv('/datasets/market_file.csv')
market_money = pd.read_csv('/datasets/market_money.csv')
market_time = pd.read_csv('/datasets/market_time.csv')
money = pd.read_csv('/datasets/money.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/market_file.csv'

In [ ]:
market_file.head()

In [ ]:
market_file.info()

In [ ]:
market_money.head()

In [ ]:
market_money.info()

In [ ]:
market_time.head()

In [ ]:
market_time.info()

In [ ]:
money.head()

In [ ]:
money = pd.read_csv('/datasets/money.csv', sep=';', decimal = ',')

In [ ]:
money.head()

In [ ]:
money.info()

__Вывод: Шаг 1.__ 
  
Загрузили 4 датасета: market_file, market_money, market_time, money.  
 - В датасете market_file необходимо привести названия столбцов к змеевому виду (русский язык оставим);  
 - Названия столбцов в датасете market_time приведем 
    к единому виду, с большой буквы;  
- Тип данных релевантен; 
 - В датасете money столбцы разделили по символу ';', разделитель десятичных значений заменили с ',' на '.'.

## __Шаг 2. Предобработка данных__

Проверим названия столбцов и приведем их к единому виду при необходимости:

In [ ]:
display(market_file.columns)
display(market_money.columns)
display(market_time.columns)
display(money.columns)

In [ ]:
market_file.columns = ['id', 
                       'Покупательская_активность', 
                       'Тип_cервиса', 
                       'Разрешить_сообщать', 
                       'Маркет_актив_6_мес', 
                       'Маркет_актив_тек_мес', 
                       'Длительность', 
                       'Акционные_покупки', 
                       'Популярная_категория',
                       'Средний_просмотр_категорий_за_визит',
                       'Неоплаченные_продукты_штук_квартал', 
                       'Ошибка_сервиса',
                       'Страниц_за_визит']

In [ ]:
market_time.columns = ['id', 'Период', 'Минут']

In [ ]:
display(market_file.columns)
display(market_time.columns)

Проверим данные на пропуски:

In [ ]:
display(market_file.isna().sum())
display(market_time.isna().sum())
display(market_money.isna().sum())
display(money.isna().sum())

Пропусков в данных не обнаружено.

Проверим данные на наличие явных дубликатов:

In [ ]:
display(market_file.duplicated().sum())
display(market_time.duplicated().sum())
display(market_money.duplicated().sum())
display(money.duplicated().sum())

Явных дубликатов не обнаружено. 
    
  
Проведем поиск неявных дубликатов:

In [ ]:
display(market_file['Покупательская_активность'].unique())
display(market_file['Тип_cервиса'].unique())
display(market_file['Разрешить_сообщать'].unique())
display(market_file['Популярная_категория'].unique())
display(market_time['Период'].unique())
display(market_money['Период'].unique())

In [ ]:
market_time['Период'] = market_time['Период'].replace(['предыдцщий_месяц'], 'предыдущий_месяц')
market_file['Тип_cервиса'] = market_file['Тип_cервиса'].replace(['стандартт'], 'стандарт')

In [ ]:
display(market_time['Период'].unique())
display(market_money['Период'].unique())
display(market_file['Тип_cервиса'].unique())

В данных были обнаруженны и исправленны неявные дубликаты.

__Вывод: Шаг 2.__ 
  
Провели предобработку данных 4 датасетов: market_file, market_money, market_time.  
 - Названия столбцов, где это было необходимо, привели к единому виду (названия всех столбцов начинаются с большой буквы, змеевой вид,  русский язык оставили);  
 - Проверили данные на пропуски, пропусков не обнаружено;  
 - Проверили данные на наличие явных и неявных дубликатов (явных дубликатов не обнаружено, от неявных дубликатов избавились).

## __Шаг 3. Исследовательский анализ данных.__
  
  
Провести исследовательский анализ данных из каждой таблицы.   
  
Отобрать клиентов с покупательской активностью не менее трех месяцев, то есть таких, которые что-либо покупали  в этот период. 
  
Оформить вывод по результатам шага.

Проанализируем таблицу market_file, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении:

In [ ]:
display(market_file.describe())
display(market_file.describe(include='object'))
display(market_file['Покупательская_активность'].value_counts())

In [ ]:
market_file['Маркет_актив_6_мес'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title ('Маркетинговая активность за последние 6 месяцев')
plt.xlabel ('Месяц')
plt.ylabel('Число рассылок, звонков, показов рекламы и прочее на клиента')
plt.show()

market_file['Маркет_актив_6_мес'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Маркетинговая активность за последние 6 месяцев')
plt.xlabel ('Месяц')
plt.ylabel('Число рассылок, звонков, показов рекламы и прочее на клиента')
plt.show()

In [ ]:
market_file['Маркет_актив_тек_мес'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title ('Маркетинговая активность за текущий месяц')
plt.xlabel ('Количество коммуникаций')
plt.ylabel('Число клиентов')
plt.show()

market_file['Маркет_актив_тек_мес'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Маркетинговая активность за текущий месяц')
plt.xlabel ('Количество коммуникаций')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Длительность'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title ('Количество дней прошедших с момента регистрации покупателя на сайте')
plt.xlabel ('Количество дней')
plt.ylabel('Число клиентов')
plt.show()

market_file['Длительность'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Количество дней прошедших с момента регистрации покупателя на сайте')
plt.xlabel ('Количество дней')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Акционные_покупки'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title ('Среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев')
plt.xlabel ('Доля акционных покупок')
plt.ylabel('Число клиентов')
plt.show()

market_file['Акционные_покупки'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев')
plt.xlabel ('Доля акционных покупок')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Средний_просмотр_категорий_за_визит'].plot(kind='hist', bins=6, grid=True, figsize=(10, 5))
plt.title ('Среднее количество категорий, которые покупатель просмотрел за визит в течение последнего месяца')
plt.xlabel ('Количество просмотренных категорий')
plt.ylabel('Число клиентов')
plt.show()

market_file['Средний_просмотр_категорий_за_визит'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Среднее количество категорий, которые покупатель просмотрел за визит в течение последнего месяца')
plt.xlabel ('Количество просмотренных категорий')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Неоплаченные_продукты_штук_квартал'].plot(kind='hist', bins=10, grid=True, figsize=(10, 5))
plt.title ('Общее число неоплаченных товаров в корзине за последние 3 месяца')
plt.xlabel ('Количество неоплаченных товаров')
plt.ylabel('Число клиентов')
plt.show()

market_file['Неоплаченные_продукты_штук_квартал'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Общее число неоплаченных товаров в корзине за последние 3 месяца')
plt.xlabel ('Количество неоплаченных товаров')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Ошибка_сервиса'].plot(kind='hist', bins=9, grid=True, figsize=(10, 5))
plt.title ('Число сбоев, которые коснулись покупателя во время посещения сайта')
plt.xlabel ('Число сбоев')
plt.ylabel('Число клиентов')
plt.show()

market_file['Ошибка_сервиса'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Число сбоев, которые коснулись покупателя во время посещения сайта')
plt.xlabel ('Число сбоев')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file['Страниц_за_визит'].plot(kind='hist', bins=20, grid=True, figsize=(10, 5))
plt.title ('Среднее количество страниц, которые просмотрел покупатель за один визит на сайте за последние 3 месяца')
plt.xlabel ('Количество страниц')
plt.ylabel('Число клиентов')
plt.show()

market_file['Страниц_за_визит'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title ('Среднее количество страниц, которые просмотрел покупатель за один визит на сайте за последние 3 месяца')
plt.xlabel ('Количество страниц')
plt.ylabel('Число клиентов')
plt.show()

In [ ]:
market_file.pivot_table(index='Покупательская_активность', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(7,7),
                                                 label='')
plt.title('Диаграмма распределения покупательской активности')
plt.show()

In [ ]:
market_file.pivot_table(index='Тип_cервиса', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(7,7),
                                                 label='')
plt.title('Диаграмма распределения типа_cервиса')
plt.show()

In [ ]:
market_file.pivot_table(index='Разрешить_сообщать', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(7,7),
                                                 label='')
plt.title('Диаграмма распределения разрешение присылать доп. предложения')
plt.show()

In [ ]:
market_file.pivot_table(index='Популярная_категория', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(8,8),
                                                 label='')
plt.title('Диаграмма распределения популярности категории товаров')
plt.show()

Проанализируем таблицу market_time с данными о времени, которое покупатель провёл на сайте:

In [ ]:
display(market_time.describe())
display(market_time.describe(include='object'))
display(market_time['Период'].value_counts())
display(market_time.groupby('Период')['Минут'].describe())

In [ ]:
market_time['Минут'].plot(kind='hist', bins=20, grid=True, figsize=(10, 5))
plt.title('Время проведенное клиентом на сайте')
plt.xlabel('Минуты')
plt.ylabel('Количество покупателей')  
plt.show()

market_time['Минут'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title('Время проведенное клиентом на сайте')
plt.xlabel('Минуты')
plt.ylabel(' ')  
plt.show()

За предыдущий и текущий месяц время пребывания клиентов на сайте не изменилось.

Построим графики сгруппировав данные по периоду:

In [ ]:
market_time.info()

In [ ]:
unique_periods_time = market_time['Период'].unique()

for period_time in unique_periods_time:
    df_period_time = market_time[market_time['Период'] == period_time]

    plt.figure(figsize=(10, 5))
    
    sns.histplot(data=df_period_time, x='Минут', kde=False, bins=15)
    
    plt.title(f'Гистограмма времени, которое покупатель провел на сайте за период: {period_time}')
    plt.xlabel('Время (минуты)')
    plt.ylabel('Количество покупателей')
    
    plt.show()

In [ ]:
market_time.pivot_table(index='Период', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(8,8))
plt.title('Период, во вроемя которого зафиксированно общее время на сайте')
plt.show()

Проанализируем таблицу market_money, которая содержит данные о выручке, которую магазин получает с покупателя:

In [ ]:
display(market_money.describe())
display(market_money.describe(include='object'))
display(market_money.groupby('Период')['Выручка'].describe())

In [ ]:
market_money['Выручка'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title('Сумма выручки за период взаимодействия')
plt.xlabel('Сумма')
plt.ylabel('Количество покупателей')  
plt.show()

market_money['Выручка'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title('Сумма выручки за период взаимодействия')
plt.xlabel('Сумма')
plt.ylabel(' ')  
plt.show()

Видим аномально большу сумму. Просмотрим подробнее.

In [ ]:
market_money.sort_values(by='Выручка', ascending=False).head(10)

От покупателя id 215380 аномально большая сумма выручки (106 862), сильно превышающая среднюю выручку (5 025). 
Возможная причина аномалии: 
- человеческий фактор (ошибка ввода суммы выручки), 
- сумма выручки действительно большая, из-за крупного заказа. 
  
Заменим сумму выручки у этого покупателя на медианную, для получения более ровных данных.


In [ ]:
market_money.loc[98,'Выручка'] = market_money['Выручка'].median()

In [ ]:
market_money.sort_values(by='Выручка', ascending=False).head(10)

In [ ]:
market_money['Выручка'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title('Сумма выручки за период взаимодействия')
plt.xlabel('Сумма')
plt.ylabel('Количество покупателей')  
plt.show()

market_money['Выручка'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title('Сумма выручки за период взаимодействия')
plt.xlabel('Сумма')
plt.ylabel(' ')  
plt.show()

In [ ]:
market_money.sort_values(by='Выручка', ascending=True).head(10)

In [ ]:
market_money.pivot_table(index='Период', 
                        aggfunc='count').plot.pie(y='id', 
                                                 autopct='%1.0f%%', 
                                                 figsize=(8,8))
plt.title('Период, во время которого зафиксированна выручка')
plt.show()

Отберем покупателей которые были активны в последние 3 месяца:

In [ ]:
market_money['Выручка'].isna().sum()

In [ ]:
market_money['Выручка'] = market_money['Выручка'].fillna(0)

In [ ]:
market_money=market_money[market_money.id.isin(market_money.query('Выручка==0')['id'].unique())==False]

In [ ]:
market_money.info()

порлучим ID клиентов с нулевой выручкой в каком либо из периодов:

In [ ]:
market_money.sort_values(by='Выручка', ascending=True).head(10)

построим графики для каждого периода:

In [ ]:
unique_periods_money = market_money['Период'].unique()

for period in unique_periods_money:
    df_period = market_money[market_money['Период'] == period]

    plt.figure(figsize=(6, 4))
    
    sns.histplot(data=df_period, x='Выручка', kde=False, bins=20)
    
    plt.title(f'Гистограмма выручки за период: {period}')
    plt.xlabel('Сумма выручки')
    plt.ylabel('Частота')
    
    plt.show()

Проанализируем таблицу money, которая содержит данные о среднемесячной прибыли продавца за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю:

In [ ]:
display(money.describe())

In [ ]:
money['Прибыль'].plot(kind='hist', bins=100, grid=True, figsize=(10, 5))
plt.title('Среднеменсячная прибыль продавца за последние 3 месяца')
plt.xlabel('Прибыль')
plt.ylabel('Количество покупателей')  
plt.show()

money['Прибыль'].plot(kind='box', vert=False, figsize=(10, 5))
plt.title('Среднеменсячная прибыль продавца за последние 3 месяца')
plt.xlabel('Прибыль')
plt.ylabel(' ')  
plt.show()

__Вывод: Шаг 3.__  

В процессе исследовательского анализа выяснилось:
- Пользовательская активность у большинства покупателей сохраняется на прежнем уровне(62%)
- Большиство покупателей выбирает стандартный тип сервиса (71%)
- Дают свое согласие на рассылку сообщений 74% пользователей, что говорит о заинтересованности покупателей в получении новых предложений.
- В тройку самых популярных групп товаров входят:   
  1) Товары для детей (25%)  
  2) Домашний текстиль (19%)  
  3) Косметика и аксессуары (17%).  
- В среднем пользователи на сайте проводщят около 13 минут (общее время проведенное на сайте за текушщий и предыдущий месяцы одинаоко).
- Выявили 1 аномально большую выручку 106862, что в 20 раз превышает среднюю выручку. Заменили это значение на медианное для устранения выбросов.Возможной причиной появления аномалии мог стать человеческий фактор (ошибочно добавлена 1 цифра) или действительно была совершена крупная покупка.
- Средняя выручка от покупателей 5025.
 Если разбить сумму средней  выручки по периодам то получим следующие цифры: 
 - Текущий месяц: 5326
 - Предыдущий месяц: 4936
 - Препредыдущий месяц: 4825
 Здесь мы видим что сумма выручки имеет тенденцию к росту из месяца в месяц.

- Отобрав клиентов с покупательской активностью не менее трех месяцев, мы получили новый датафрей в котором 1297 пользователей. Это значит, что всего 3 пользователя из исходного датафрейма market_money не подошли нам по критерию.

## __Шаг 4. Объединение таблиц.__  
Объединить таблицы market_file, market_money, market_time.
Учитывая, что данные о выручке и времени на сайте находятся в одном столбце для всех периодов. В итоговой таблице сделаем отдельный столбец для каждого периода

In [ ]:
market_time.info()

In [ ]:
market_time_1=market_time[market_time['Период']=='предыдущий_месяц'].copy()

In [ ]:
market_time_1.rename(columns={'Минут':'Минут_предыдущего_месяца'},inplace=True)

In [ ]:
market_time_1.drop('Период', axis= 1, inplace= True )

In [ ]:
market_time_2= market_time[market_time['Период']=='текущий_месяц'].copy()

In [ ]:
market_time_2.rename(columns={'Минут':'Минут_текущего_месяца'},inplace=True)

In [ ]:
market_time_2.drop('Период', axis= 1 , inplace= True )

In [ ]:
market_time_fin = pd.merge(market_time_1, market_time_2, on='id', sort=True)

In [ ]:
market_time_fin.info()

In [ ]:
market_time_fin.head()

In [ ]:
market_money.info()

In [ ]:
market_money.head()

In [ ]:
market_money_1 = market_money[market_money['Период']=='текущий_месяц'].copy()

In [ ]:
market_money_1.rename(columns={'Выручка':'Выручка_текущий_месяц'},inplace=True)

In [ ]:
market_money_1.drop('Период', axis= 1 , inplace= True )

In [ ]:
market_money_2 = market_money[market_money['Период']=='предыдущий_месяц'].copy()

In [ ]:
market_money_2.rename(columns={'Выручка':'Выручка_предыдущий_месяц'},inplace=True)

In [ ]:
market_money_2.drop('Период', axis= 1 , inplace= True )

In [ ]:
market_money_3 = market_money[market_money['Период']=='препредыдущий_месяц'].copy()

In [ ]:
market_money_3.rename(columns={'Выручка':'Выручка_препредыдущий_месяц'},inplace=True)

In [ ]:
market_money_3.drop('Период', axis= 1 , inplace= True )

In [ ]:
market_money_fin = pd.merge(market_money_1, market_money_2, on='id', sort=True)

In [ ]:
market_money_fin=pd.merge(market_money_fin, market_money_3, on='id', sort=True)

In [ ]:
market_money_fin.info()

In [ ]:
market_money_fin.head()

In [ ]:
market_file.info()

In [ ]:
total_df= pd.merge(market_file, pd.merge(market_time_fin, 
         market_money_fin, 
         on='id', sort=True), on='id', sort=True)

In [ ]:
total_df.info()

In [ ]:
total_df.head()

__Вывод: Шаг 4__    
Успешно объединили 3 датафрейма market_file, market_money, market_time, разделив время и выручку по периодам. 

## __Шаг 5. Корреляционный анализ__. 
Проведем корреляционный анализ признаков в количественной шкале в итоговой таблице для моделирования. Сделаем выводы о мультиколлинеарности и при необходимости устраним её.

In [ ]:
total_df.info()

In [ ]:
len(total_df['id'].unique())

In [ ]:
numeric_cols = total_df.select_dtypes(include=['number']).columns
correlation_matrix = total_df[numeric_cols].corr(method='spearman')

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

__Вывод: Шаг 5.__   
  
В данных прослеживается высокая кореляция только между признаками "выручка за текущий месяц" и "выручка за предыдущий месяц": 0,88. Это указывает на то, что клиенты, тратящие больше в один месяц, часто продолжают тратить аналогичные суммы в другие месяцы.

В целом, корреляции между признаками из разных категорий (например, маркетинговая активность, выручка, поведение на сайте) остаются слабыми.

Так как отсутсвует очень высокая связь между парметрами, мы можем не учитывать мультиколениальность.

## __Шаг 6. Использование пайплайнов__.     

Применим все изученные модели. Для этого используйте пайплайны.

6.1 Во время подготовки данных используем ColumnTransformer. Количественные и категориальные признаки обработаем в пайплайне раздельно. Для кодирования категориальных признаков используем как минимум два кодировщика, для масштабирования количественных — как минимум два скейлера. Для каждой модели можно подготовить данные с разным кодированием и масштабированием.

6.2 Обучим четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и  SVC(). Для каждой из них подберем как минимум один гиперпараметр. Выберем подходящую для задачи метрику. Используем эту метрику при подборе гиперпараметров.
   
6.3 Выберем лучшую модель, используя заданную метрику. Для этого применим одну из стратегий:
- использовать пайплайны и инструменты подбора гиперпараметров для каждой модели отдельно, чтобы выбрать лучшую модель самостоятельно;
- использовать один общий пайплайн для всех моделей и инструмент подбора гиперпараметров, который вернёт вам лучшую модель.

In [ ]:
X = total_df.drop(['id', 
                   'Покупательская_активность'], axis=1)
y = total_df['Покупательская_активность']
y= y.apply(lambda x:0 if x=='Прежний уровень' else 1)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=TEST_SIZE, 
                                                    random_state=RANDOM_STATE,
                                                    stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
ohe_columns = ['Разрешить_сообщать',                  
               'Популярная_категория']
ord_columns = ['Тип_cервиса']
num_columns = ['Выручка_текущий_месяц',
               'Выручка_предыдущий_месяц',
               'Выручка_препредыдущий_месяц',
               'Минут_предыдущего_месяца',
               'Минут_текущего_месяца',
               'Маркет_актив_6_мес',
               'Маркет_актив_тек_мес',
               'Длительность',
               'Акционные_покупки',
               'Средний_просмотр_категорий_за_визит',
               'Неоплаченные_продукты_штук_квартал',             
               'Ошибка_сервиса',            
               'Страниц_за_визит']

In [ ]:
ohe_pipe = Pipeline(
    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ohe', OneHotEncoder(drop='first', handle_unknown='error', sparse=False))
    ]
    )

ord_pipe = Pipeline(
    [('simpleImputer_before_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ord', OrdinalEncoder(
         categories=[
             ['премиум', 'стандарт']
         ], 
         handle_unknown='use_encoded_value', unknown_value=np.nan
     )
     ),
     ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ]
)

data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', StandardScaler(), num_columns)
    ], 
    remainder='passthrough'
)

Модель KNeighbors Classifier

In [ ]:
# создаём итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', KNeighborsClassifier())
])

param_grid = [
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        
    }
]

knc = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

knc.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', knc.best_estimator_)
print ('Метрика ROC-AUC лучшей модели на кросс-валидации :', knc.best_score_)

In [ ]:
choosing_model_separately = pd.DataFrame(knc.cv_results_).sort_values(
    by=['rank_test_score']
).head(1)
choosing_model_separately[['rank_test_score', 'param_models', 'mean_test_score', 'params']]

Модель Decision Tree

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 700),
        'models__max_features': range(2, 16),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        
    }
]

dtc = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

dtc.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', dtc.best_estimator_)
print ('Метрика ROC-AUC лучшей модели на кросс-валидации:', dtc.best_score_)

In [ ]:
choosing_model_1 = pd.DataFrame(dtc.cv_results_).sort_values(by=['rank_test_score']).head(1)
choosing_model_separately = pd.concat([choosing_model_separately, choosing_model_1], ignore_index= True )
choosing_model_separately[['rank_test_score', 'param_models', 'mean_test_score', 'params']]

Модель Logistic Regression

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1'
        )],
        'models__C': range(1, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        
    }
]

lr = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

lr.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', lr.best_estimator_)
print ('Метрика ROC-AUC лучшей модели на кросс-валидации:', lr.best_score_)

In [ ]:
choosing_model_1 = pd.DataFrame(lr.cv_results_).sort_values(by=['rank_test_score']).head(1)
choosing_model_separately = pd.concat([choosing_model_separately, choosing_model_1], ignore_index= True )
choosing_model_separately[['rank_test_score', 'param_models', 'mean_test_score', 'params']]

Модель SVC

pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', SVC(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [SVC(random_state=RANDOM_STATE, probability=True)],
        'models__kernel': ['poly', 'rbf', 'sigmoid'],
        'models__degree': range(2, 10),
        'preprocessor__num': [StandardScaler(), 
                              MinMaxScaler(), 
                              RobustScaler(), 
                              'passthrough'],
        
    }
]

svc = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

svc.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', svc.best_estimator_)
print ('Метрика ROC-AUC лучшей модели на кросс-валидации :', svc.best_score_)

choosing_model_1 = pd.DataFrame(svc.cv_results_).sort_values(by=['rank_test_score']).head(1)
choosing_model_separately = pd.concat([choosing_model_separately, choosing_model_1], ignore_index= True )
choosing_model_separately[['rank_test_score', 'param_models', 'mean_test_score', 'params']].sort_values(by=['mean_test_score'], ascending = False)


Проведем финальное тестирование 

y_test_pred=svc.predict_proba(X_test)[:, 1]

print('Метрика ROC-AUC лучшей модели на тестовой выборке:', {roc_auc_score(y_test, y_test_pred)})

__Вывод Шаг 6__

Лучшей моделью оказалась SVC(0.914).  

KNeighborsClassifier (0.905),   
Decision Tree (0.808),   
Logistic Regression (0.897)

Значения ROC-AUC выше 0.9 говорят о том, что модель очень хорошо разделяет классы.

## __Шаг 7. Анализ важности признаков__
  
7.1 Оценим важность признаков для лучшей модели и построим график важности с помощью метода SHAP.   

7.2 Сделаем выводы о значимости признаков:
какие признаки мало значимы для модели;
какие признаки сильнее всего влияют на целевой признак;
как можно использовать эти наблюдения при моделировании и принятии бизнес-решений.

In [ ]:
COUNT = 10

best_model =  svc.best_estimator_.named_steps['models']

preprocessor = svc.best_estimator_.named_steps['preprocessor'] 

#preprocessor = svc.best_estimator_.named_steps['preprocessor'].get_feature_names_out() 
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

ohe_encoder = preprocessor.named_transformers_['ohe'].named_steps['ohe']

# Проверяем доступные методы в OneHotEncoder
if hasattr(ohe_encoder, "get_feature_names_out"):
    ohe_feature_names = ohe_encoder.get_feature_names_out(input_features=ohe_columns)
else:  # Для старых версий sklearn
    categories = ohe_encoder.categories_
    ohe_feature_names = [f"{col}_{val}" for col, vals in zip(ohe_columns, categories) for val in vals[1:]]

# Объединяем имена признаков

ord_feature_names = ord_columns
num_feature_names = num_columns
all_feature_names = np.concatenate([ohe_feature_names, ord_feature_names, num_feature_names])

# Преобразуем обратно в DataFrame
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=all_feature_names)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=all_feature_names)

# Выбираем небольшую выборку
X_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, COUNT, random_state=RANDOM_STATE)
X_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, COUNT, random_state=RANDOM_STATE)

# Проверяем размеры
print(f"X_train_preprocessed_smpl shape: {X_train_preprocessed_smpl.shape}")
print(f"X_test_preprocessed_smpl shape: {X_test_preprocessed_smpl.shape}")


In [ ]:
explainer_2 = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
shap_values_2 = explainer_2.shap_values(X_test_preprocessed_smpl, nsamples=100)
shap.initjs()
shap.force_plot(explainer_2.expected_value[0], shap_values_2[..., 0], X_test)


In [ ]:
explainer = shap.Explainer(best_model.predict_proba, X_train_preprocessed_smpl)

shap_values = explainer(X_test_preprocessed_smpl)

shap.summary_plot(shap_values[..., 1], X_test_preprocessed_smpl, feature_names=all_feature_names)
shap.summary_plot(shap_values[..., 1], X_test_preprocessed_smpl, feature_names=all_feature_names, plot_type='bar')

__Вывод Шаг 7__

 Сильнее всего вляют на целевой признак: 
 1) Акционные_покупки,  
 2) Страниц_за_визит,  
 3) Минут_текущего_месяца,  
 4) Выручка_пердыдущего_месяца,  
 5) Минут_предыдущего_месяца.
 
 Мало значимые признаки для модели: 
 1) Популярна_категория_Кухонная_посуда,  
 2) Тип_сервиса,  
 3) Популярная_категория_Товары для детей,  
 4) Разрешить_сообщать_нет,  
 5) Ошибка_сервиса  
   
Чтобы бизнесу сфокусироваться на удержании и мотивации активных пользователей, а также на стимулировании повторных покупок стоит уделить серьезное внимание разработатке стратегии и активного продвижения акционных и персональных предложений для постоянных покупателей. 
     
Учитывая влияние времени, проведенного на сайте, стоит улучшить навигацию, фильтры и рекомендации, чтобы стимулировать пользователей проводить больше времени в сервисе.
  
Поскольку количество страниц за визит значимо, можно экспериментировать с динамическим контентом, показывая релевантные категории и продукты, увеличивающие вовлеченность.
  
Так как выручка за предыдущие месяцы влияет на активность, можно использовать программы лояльности, напоминающие клиентам о товарах, которые они покупали ранее.


## __Шаг 8. Сегментация покупателей:__  

8.1 Выполним сегментацию покупателей. Используя результаты моделирования и данные о прибыльности покупателей.  

8.2 Выберем группу покупателей и предложим, как увеличить её покупательскую активность:     
- Проведем графическое и аналитическое исследование группы покупателей.  
- Сделаем предложения по работе с сегментом для увеличения покупательской активности.  

8.3 Сделаем выводы о сегментах:    
- какой сегмент мы взяли для дополнительного исследования,  
- какие предложения мы сделали и почему.

In [ ]:
preprocessor = svc.best_estimator_.named_steps['preprocessor']

In [ ]:
X_transformed = preprocessor.transform(X)
X_transformed = pd.DataFrame(X_transformed)

In [ ]:
#добавим вероятность снижения
total_df['Вероятность_снижения'] = svc.best_estimator_.named_steps['models'].predict_proba(X_transformed)[:, 1]

In [ ]:
#добавим в датафрейм выручку клиентов за 3 месяца
total_df['Суммарная_выручка'] = (
    total_df['Выручка_текущий_месяц'] + 
    total_df['Выручка_предыдущий_месяц'] + 
    total_df['Выручка_препредыдущий_месяц']
)

In [ ]:
#Разделим покупателей по вероятности ухода (используем 0.5 как порог):
total_df['Риск_оттока'] = total_df['Вероятность_снижения'].apply(lambda x: 'Высокий' if x > 0.5 else 'Низкий')

In [ ]:
#Определим уровень прибыльности, разделив клиентов на три группы по квартилям:
total_df['Категория_прибыльности'] = pd.qcut(total_df['Суммарная_выручка'], q=3, labels=['Низкая', 'Средняя', 'Высокая'])

In [ ]:
#разделим клиентов по сегентам
def assign_segment(row):
    if row['Риск_оттока'] == 'Низкий' and row['Категория_прибыльности'] == 'Высокая':
        return 'Лояльные VIP'
    elif row['Риск_оттока'] == 'Низкий':
        return 'Перспективные'
    elif row['Риск_оттока'] == 'Высокий' and row['Категория_прибыльности'] == 'Высокая':
        return 'Рискованные VIP'
    else:
        return 'Отходящие'

total_df['Сегмент'] = total_df.apply(assign_segment, axis=1)

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=total_df, x='Сегмент', order=['Лояльные VIP', 'Перспективные', 'Рискованные VIP', 'Отходящие'])
plt.xticks(rotation=45)
plt.title('Распределение клиентов по сегментам')
plt.xlabel('Сегмент')
plt.ylabel('Количество клиентов')
plt.show()

На визуализации мы видим что большинство покупателей находятся в сегменте "Перспективные", а "Лояльные VIP" и "Отходящие" в одинаковом количестве. 

Для детального анализа возьмем сегмент "Рискованные VIP", так как у него высокая категория прибыльности, что делает этот сегмент ключевым для бизнеса.

In [ ]:
# Фильтрация двух сегментов
risky_vip = total_df[total_df["Сегмент"] == "Рискованные VIP"]
loyal_vip = total_df[total_df["Сегмент"] == "Лояльные VIP"]
perspect = total_df[total_df["Сегмент"] == "Перспективные"]
othod = total_df[total_df["Сегмент"] == "Отходящие"]

In [ ]:
# Описание сегмента
print("Описание сегмента 'Рискованные VIP':")
print(risky_vip.describe())
print("\nРаспределение категориальных признаков:")
print(risky_vip.describe(include=['object', 'category']))

In [ ]:
# График распределения вероятности снижения покупательской активности
plt.figure(figsize=(10, 6))
sns.histplot(risky_vip["Вероятность_снижения"], bins=20, kde=True, color='red')
plt.title("Распределение вероятности снижения покупательской активности среди 'Рискованных VIP'")
plt.xlabel("Вероятность снижения")
plt.ylabel("Количество покупателей")
plt.show()

In [ ]:
# Анализ выручки
plt.figure(figsize=(10, 6))
sns.boxplot(x=risky_vip["Категория_прибыльности"], y=risky_vip["Суммарная_выручка"], palette="coolwarm")
plt.title("Распределение суммарной выручки по категориям прибыльности среди 'Рискованных VIP'")
plt.xlabel("Категория прибыльности")
plt.ylabel("Суммарная выручка")
plt.show()

Выполним сравнительный анализ сегментов "Рискованных VIP" и "Лояльные VIP"

In [ ]:
# Сравнительные статистики
stats_columns = [
    "Суммарная_выручка",
    "Выручка_текущий_месяц",
    "Выручка_предыдущий_месяц",
    "Минут_текущего_месяца",
    "Минут_предыдущего_месяца",
    "Вероятность_снижения",
    "Маркет_актив_6_мес",
    "Маркет_актив_тек_мес",
    "Акционные_покупки",
    "Страниц_за_визит"
       
]

summary_stats = pd.DataFrame({
    'Метрика': stats_columns,
    'Рискованные VIP': [risky_vip[m].mean() for m in stats_columns],
    'Лояльные VIP': [loyal_vip[m].mean() for m in stats_columns],
    'Перспективные': [perspect[m].mean() for m in stats_columns],
    'Отходящие': [othod[m].mean() for m in stats_columns]
})

In [ ]:
summary_stats.head()

In [ ]:
# Список переменных для визуализации
features = [
    ("Маркет_актив_6_мес", "Маркетинговая активность за 6 мес"),
    ("Маркет_актив_тек_мес", "Маркетинговая активность в текущем месяце"),
    ("Суммарная_выручка", "Суммарная выручка"),
    ("Акционные_покупки", "Акционные покупки"),
    ("Страниц_за_визит", "Просмотренные страницы за визит"),
    ("Минут_текущего_месяца", "Время в текущем месяце (минуты)"),
    ("Минут_предыдущего_месяца", "Время в предыдущем месяце (минуты)"),
    ("Выручка_предыдущий_месяц", "Выручка предыдущего месяца")
]

# Визуализация с помощью цикла
plt.figure(figsize=(12, len(features) * 4))  # Настройка размера общей фигуры

for i, (feature, title) in enumerate(features, 1):
    plt.subplot(len(features), 1, i)  # Создание подграфиков
    sns.boxplot(data=total_df, x="Сегмент", y=feature, palette=["red", "blue", "green", "yellow" ])
    plt.xlabel("Сегмент")
    plt.ylabel(feature)
    plt.title(f"Распределение {title} в Сегментах")

plt.tight_layout()  # Уплотнение графиков для лучшего отображения
plt.show()

__Вывод Шаг 8.__
  
По сравнительным характеристикам видим, что сегмент "Рискованные VIP" заметно меньше проводят времени на сайте и посещают меньше страниц чем другие категории, но при этом оставляют достаточно высокую выручку и активно покупают акционные товары.

Исходя из анализа, можно дать следующие рекомендации для данного сегмента, чтобы повысить их удовлетворенность и стимулировать к увеличению объема покупок через таргетированные и привлекательные предложения.

  
1. Необходимо усиление акционных предложений:
- Персонализированные скидки: Предоставлять индивидуальные скидки на товары и услуги, которые соответствуют интересам и предыдущим покупкам клиентов.
- Эксклюзивные акции: Разрабатывать специальные предложения, доступные только для этого сегмента, чтобы повысить их лояльность и стимулировать повторные покупки.

  
2. Разработать программы лояльности:
- Накопительные бонусы: Вводить системы бонусных баллов за каждую покупку, которые можно обменять на товары или услуги.
- Ранний доступ: Предоставлять "Рискованным VIP" клиентам возможность первыми участвовать в распродажах или получать доступ к новым продуктам.

  
3. Улучшить коммуникацию:
- Делать персонализированные рассылки: Отправлять предложения и новости, основанные на предпочтениях и поведении клиентов.
- Уделять внимание обратной связи: Активно собирать мнения и пожелания клиентов для улучшения сервиса и предложения более релевантных акций.
  
  
4. Регулярно проводить анализ поведения:
- Вести мониторинг откликов: Отслеживать, какие акции наиболее эффективны для данного сегмента, и адаптировать стратегию в соответствии с полученными данными.
- Предсказывать отток: Использовать аналитические инструменты для выявления признаков возможного снижения активности и принимать превентивные меры.

## __Шаг 9. Общий вывод__

В рамках данного проекта для интернет-магазина «В один клик» была проделана комплексная работа по анализу данных, построению модели для прогнозирования активности клиентов, сегментации пользователей и разработке рекомендаций для увеличения покупательской активности. 

Цель проекта состояла в удержании существующих клиентов за счёт персонализированных предложений, основанных на их покупательской активности и прибыльности.

Выполнили следующие шаги: 
1. Загрузили данные и проверили их что данные в таблицах соответствуют описанию.
2. Провели необходимую предобработку данных.
3. Провели исследовательский анализ данных из каждой таблицы.
4. Объединили таблицы market_file, market_money, market_time.
5. Провели корреляционный анализ признаков.
6. Нашли лучшую модель. Для этого использовали пайплайны. Ей оказалась SVC(degree=5, probability=True, random_state=42). 
7. Провели анализ важности признаков.
Малозначимыми признаки для модели оказались:
 1) Популярна_категория_Кухонная_посуда,  
 2) Тип_сервиса,  
 3) Популярная_категория_Товары для детей,  
 4) Разрешить_сообщать_нет,  
 5) Ошибка_сервиса 
Сильнее всего влияют на целевой признак:
 1) Акционные_покупки,  
 2) Страниц_за_визит,  
 3) Минут_текущего_месяца,  
 4) Выручка_пердыдущего_месяца,  
 5) Минут_предыдущего_месяца.


8. Выполнили сегментацию покупателей, в результате чего смогли дать  определенные рекомендации для сегмента "Рискованные VIP" чтобы повысить их удовлетворенность и стимулировать к увеличению объема покупок через таргетированные и привлекательные предложения.:
  
1. Необходимо усиление акционных предложений:
- Персонализированные скидки: Предоставлять индивидуальные скидки на товары и услуги, которые соответствуют интересам и предыдущим покупкам клиентов.
- Эксклюзивные акции: Разрабатывать специальные предложения, доступные только для этого сегмента, чтобы повысить их лояльность и стимулировать повторные покупки.

  
2. Разработать программы лояльности:
- Накопительные бонусы: Вводить системы бонусных баллов за каждую покупку, которые можно обменять на товары или услуги.
- Ранний доступ: Предоставлять "Рискованным VIP" клиентам возможность первыми участвовать в распродажах или получать доступ к новым продуктам.

  
3. Улучшить коммуникацию:
- Делать персонализированные рассылки: Отправлять предложения и новости, основанные на предпочтениях и поведении клиентов.
- Уделять внимание обратной связи: Активно собирать мнения и пожелания клиентов для улучшения сервиса и предложения более релевантных акций.
  
  
4. Регулярно проводить анализ поведения:
- Вести мониторинг откликов: Отслеживать, какие акции наиболее эффективны для данного сегмента, и адаптировать стратегию в соответствии с полученными данными.
- Предсказывать отток: Использовать аналитические инструменты для выявления признаков возможного снижения активности и принимать превентивные меры.


Внедрение предложенных мер поможет интернет-магазину «В один клик» удерживать клиентов с высоким показателем выручки но высоким риском оттока и повысить их лояльность к бренду.
